In [378]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import numpy as np 

def getSoupFromUrl(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    return soup

places = ["aquitaine","ile_de_france","provence_alpes_cote_d_azur"]
l = []

for place in places:
    url = "http://www.leboncoin.fr/voitures/offres/"+place+"/?f=a&th=1&q=Renault+Zoé"
    soup = getSoupFromUrl(url)
    print "this is for "+ str(place)
    allcars = soup.findAll("div",{"class":"list-lbc"})
    #print type(allcars)
    links = allcars[0].findAll('a')
    #print links
    for link in links:
        href = link.get('href')
        print "------------"
        #print href
        internsoup = getSoupFromUrl(href)  
        carsprice = internsoup.find("div",{"class":"floatLeft"})
        for row in carsprice.find('table').tbody.findAll('tr'):
            if  row.find('span') is not None:
                price_ = int(row.find('span').text.replace(' ','').replace(u"\u20AC",''))   
                print price_
                #print type(price)
        carsinfo = internsoup.find("div",{"class":"lbcParams criterias"})
        for row in carsinfo.find('table').findAll('tr'):
            infos = row.find('td').text
            kilo = re.search('.* (KM)', infos, re.I) 
            if kilo:
                kilo = int(kilo.group(0).replace('KM','').replace(' ',''))
                #print kilo
                kilometrage = kilo
                #print type(kilo)
        print kilometrage
        releasedate = int(internsoup.find('td',{"itemprop":"releaseDate"}).text.strip())
        print  releasedate
        #print type(releasedate)
        poster = internsoup.find_all("div", {"class":"upload_by"})
        pro = poster[0].find_all("span", {"class":"ad_pro"})
        if len(pro) == 1: 
            pro_or_part = "professionel"
        else: 
            pro_or_part = "particulier"
        print pro_or_part
        description = internsoup.find("div",{"class":"content"}).text.strip().replace('\n', ' ') 
        #print description
        version = re.search('ZEN|LIFE|INTENS',description, re.I)
        if version :
            version = version.group(0)
            print version
        phonenumber = re.search('^.*\D?(0[0-9].?([0-9][0-9].?){4})\D?.*',description)
        if phonenumber:
            phonenumber = phonenumber.group(1)
            print phonenumber
            l.append([kilometrage,price_,releasedate,pro_or_part,version,phonenumber])
    print "=================="
    
    

this is for aquitaine
------------
12000
18600
2013
particulier
Intens
06 81 31 28 45.
------------
10000
30000
2013
particulier
zen
------------
12000
11000
2013
particulier
------------
12500
14000
2013
particulier
ZEN
this is for ile_de_france
------------
14490
15000
2014
professionel
06.61.71.36.89R
------------
12990
3299
2014
professionel
Life
0805805802
------------
15490
2250
2014
professionel
Intens
0805805802
------------
15490
10
2014
professionel
Intens
0805805802
------------
15490
10
2014
professionel
Intens
0805805802
------------
15490
2250
2014
professionel
Intens
0805805802
------------
9500
31000
2013
particulier
ZEN
------------
12890
18032
2014
professionel
Life
0609775076
------------
9200
16100
2013
particulier
Life
------------
13500
12000
2014
particulier
INTENS
------------
10990
16515
2013
professionel
ZEN
0175448163
------------
12990
3299
2014
professionel
Life
0805805802
------------
12990
1000
2015
particulier
Life
------------
17900
235
2015
professione

In [379]:
df1 = pd.DataFrame(l, columns=["kilometrage", "price", "release_date", "pro_or_part", "version","phone_number"])  
print df1

    kilometrage  price  release_date   pro_or_part version     phone_number
0         18600  12000          2013   particulier  Intens  06 81 31 28 45.
1         15000  14490          2014  professionel    None  06.61.71.36.89R
2          3299  12990          2014  professionel    Life       0805805802
3          2250  15490          2014  professionel  Intens       0805805802
4            10  15490          2014  professionel  Intens       0805805802
5            10  15490          2014  professionel  Intens       0805805802
6          2250  15490          2014  professionel  Intens       0805805802
7         18032  12890          2014  professionel    Life       0609775076
8         16515  10990          2013  professionel     ZEN       0175448163
9          3299  12990          2014  professionel    Life       0805805802
10          235  17900          2015  professionel  Intens       0677372750
11        26454   8990          2013  professionel    LIFE       0174704406
12        28

In [380]:
cote_prices = []
for index, row in df1.iterrows():
     if (row['version'] is None):
            cote_prices.append(0)
     else:
        a = row['release_date']
        b = row['version']
        url = "http://www.lacentrale.fr/cote-auto-renault-zoe-"+str(b)+"+charge+rapide-"+str(a)+".html"
        cote_soup = getSoupFromUrl(url)
        cote_price = cote_soup.find("span",{"class":"Result_Cote arial tx20"}).text.replace(' ','').replace(u"\u20AC",'')
        cote_prices.append(int(cote_price))
print cote_prices            

[0, 12000, 13800, 13800, 13800, 13800, 10700, 12000, 10400, 13800, 10700, 12000, 0, 0, 10400, 11100, 11100, 10400, 12400]


In [381]:
df2 = pd.DataFrame(cote_prices, columns=["Cote_price"])
print df2          

    Cote_price
0            0
1        12000
2        13800
3        13800
4        13800
5        13800
6        10700
7        12000
8        10400
9        13800
10       10700
11       12000
12           0
13           0
14       10400
15       11100
16       11100
17       10400
18       12400


In [382]:
# Concaténation des deux dataframes
allinfos_dataframe = pd.concat([df1, df2], axis=1)
print allinfos_dataframe

    kilometrage  price  release_date   pro_or_part version     phone_number  \
0         18600  12000          2013   particulier  Intens  06 81 31 28 45.   
1         15000  14490          2014  professionel    None  06.61.71.36.89R   
2          3299  12990          2014  professionel    Life       0805805802   
3          2250  15490          2014  professionel  Intens       0805805802   
4            10  15490          2014  professionel  Intens       0805805802   
5            10  15490          2014  professionel  Intens       0805805802   
6          2250  15490          2014  professionel  Intens       0805805802   
7         18032  12890          2014  professionel    Life       0609775076   
8         16515  10990          2013  professionel     ZEN       0175448163   
9          3299  12990          2014  professionel    Life       0805805802   
10          235  17900          2015  professionel  Intens       0677372750   
11        26454   8990          2013  professionel  

In [383]:
for index, row in allinfos_dataframe.iterrows():
    if ( row["Cote_price"] < row["price"]):
        allinfos_dataframe.set_value(index, "comparison", "expensive car")
    else:
        allinfos_dataframe.set_value(index, "comparison", " great deal ")
print allinfos_dataframe

    kilometrage  price  release_date   pro_or_part version     phone_number  \
0         18600  12000          2013   particulier  Intens  06 81 31 28 45.   
1         15000  14490          2014  professionel    None  06.61.71.36.89R   
2          3299  12990          2014  professionel    Life       0805805802   
3          2250  15490          2014  professionel  Intens       0805805802   
4            10  15490          2014  professionel  Intens       0805805802   
5            10  15490          2014  professionel  Intens       0805805802   
6          2250  15490          2014  professionel  Intens       0805805802   
7         18032  12890          2014  professionel    Life       0609775076   
8         16515  10990          2013  professionel     ZEN       0175448163   
9          3299  12990          2014  professionel    Life       0805805802   
10          235  17900          2015  professionel  Intens       0677372750   
11        26454   8990          2013  professionel  

In [360]:
#j'écris mon dataframe dans un fichier 
allinfos_dataframe.to_csv("cars.csv", sep=';')